In [1]:
#Importing libaries
import pandas as pd
import os
import dask.dataframe as dd
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import numpy as np
from pathlib import Path

**Exploring the Local Strom Report and National Flood Insturance Program (NFIP) database**

In [2]:
#Directory of the LSR and pluvial claim dataset
LSR = Path("../data/CONUS/LSR_pluvial_CONUS_X.csv")
pluvial_claims = Path("../data/CONUS/Claim_CONUS_2005_X.csv")

output_path = "../data/CONUS/Claim_CONUS_2005_X_withdamage.csv"
#Out put dir
out_dir = "../data/CONUS/LSR_pluvial_CONUS_X_damagepercent.csv"

In [3]:
df_lsr = pd.read_csv(LSR)
df_lsr.columns
#Note: The FIPS is the census tract

Index(['VALID', 'VALID2', 'LAT', 'LON', 'MAG', 'WFO', 'TYPECODE', 'TYPETEXT',
       'CITY', 'COUNTY', 'STATE', 'SOURCE', 'REMARK', 'UGC', 'UGCNAME',
       'QUALIFIER', 'OBJECTID', 'Join_Count', 'TARGET_FID', 'STATE_ABBR',
       'STATE_FIPS', 'COUNTY_FIPS', 'STCOFIPS', 'TRACT_FIPS', 'FIPS',
       'POPULATION', 'POP_SQMI', 'SQMI', 'POPULATION_2020', 'POP20_SQMI'],
      dtype='object')

In [4]:
df_pluvialclaims = pd.read_csv(pluvial_claims)
df_pluvialclaims.columns

Index(['agricultureStructureIndicator', 'asOfDate', 'baseFloodElevation',
       'basementEnclosureCrawlspace', 'reportedCity', 'condominiumIndicator',
       'policyCount', 'countyCode', 'communityRatingSystemDiscount',
       'dateOfLoss', 'elevatedBuildingIndicator',
       'elevationCertificateIndicator', 'elevationDifference', 'censusTract',
       'floodZone', 'houseWorship', 'latitude', 'locationOfContents',
       'longitude', 'lowestAdjacentGrade', 'lowestFloorElevation',
       'numberOfFloorsInTheInsuredBuilding', 'nonProfitIndicator',
       'obstructionType', 'occupancyType', 'originalConstructionDate',
       'originalNBDate', 'amountPaidOnBuildingClaim',
       'amountPaidOnContentsClaim',
       'amountPaidOnIncreasedCostOfComplianceClaim',
       'postFIRMConstructionIndicator', 'rateMethod',
       'smallBusinessIndicatorBuilding', 'state',
       'totalBuildingInsuranceCoverage', 'totalContentsInsuranceCoverage',
       'yearOfLoss', 'reportedZipcode', 'primaryReside

**Getting the damage (%) from NFIP data and based on the census tract boundary, adding the damage % column into the LSR event database**

In [ ]:
def calculate_damage_percent(df):
    df = df.copy()

    # Filling NaNs with 0 for relevant columns
    cols = ['amountPaidOnBuildingClaim', 'amountPaidOnContentsClaim',
            'totalBuildingInsuranceCoverage', 'totalContentsInsuranceCoverage']
    df[cols] = df[cols].fillna(0)

    # Initialize damagePercent column
    df['damagePercent'] = np.nan

    # Both building and contents valid
    both_mask = (
        (df['totalBuildingInsuranceCoverage'] > 0) &
        (df['totalContentsInsuranceCoverage'] > 0) &
        (df['amountPaidOnBuildingClaim'] > 0) &
        (df['amountPaidOnContentsClaim'] > 0)
    )
    df.loc[both_mask, 'damagePercent'] = (
        df['amountPaidOnBuildingClaim'] + df['amountPaidOnContentsClaim']
    ) / (
        df['totalBuildingInsuranceCoverage'] + df['totalContentsInsuranceCoverage']
    ) * 100

    # Only building info
    building_mask = (
        ((df['totalContentsInsuranceCoverage'] == 0) | (df['amountPaidOnContentsClaim'] == 0)) &
        (df['totalBuildingInsuranceCoverage'] > 0) &
        (df['amountPaidOnBuildingClaim'] > 0)
    )
    df.loc[building_mask, 'damagePercent'] = (
        df['amountPaidOnBuildingClaim'] / df['totalBuildingInsuranceCoverage']
    ) * 100

    # Only contents info
    contents_mask = (
        ((df['totalBuildingInsuranceCoverage'] == 0) | (df['amountPaidOnBuildingClaim'] == 0)) &
        (df['totalContentsInsuranceCoverage'] > 0) &
        (df['amountPaidOnContentsClaim'] > 0)
    )
    df.loc[contents_mask, 'damagePercent'] = (
        df['amountPaidOnContentsClaim'] / df['totalContentsInsuranceCoverage']
    ) * 100

    # Cap at 100%
    df['damagePercent'] = df['damagePercent'].clip(upper=100)

    # Convert dateOfLoss to datetime
    df['dateOfLoss'] = pd.to_datetime(df['dateOfLoss'], errors='coerce')

    # Compute groupwise mean
    group_means = df.groupby(['censusTract', 'dateOfLoss'])['damagePercent'].mean().reset_index(name='meanDamagePercent')

    # Take one representative row from each group
    first_rows = df.sort_values('dateOfLoss').groupby(['censusTract', 'dateOfLoss']).first().reset_index()

    # Merge mean damage percent
    result = first_rows.merge(group_means, on=['censusTract', 'dateOfLoss'], how='left')

    return result


In [6]:
# Add damagePercent column
df_with_damage = calculate_damage_percent(df_pluvialclaims)

# Export to CSV
df_with_damage.to_csv(output_path, index=False)

print(f"Exported to: {output_path}")

Exported to: ../data/CONUS/Claim_CONUS_2005_X_withdamage.csv


In [7]:
#The Updated dataset
df_nfip_with_damage = pd.read_csv(output_path)
df_nfip_with_damage.describe()

,censusTract,agricultureStructureIndicator,baseFloodElevation,basementEnclosureCrawlspace,policyCount,countyCode,communityRatingSystemDiscount,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,...,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,reportedZipcode,primaryResidence,causedBy100yr,damagePercent,meanDamagePercent
count,4.995400e+04,49954.000000,80.000000,30879.000000,49954.000000,49954.000000,20089.000000,49954.000000,0.0,108.000000,...,47039.000000,49954.000000,4.995400e+04,49954.000000,49954.000000,49950.000000,49954.000000,49954.0,36381.000000,36381.000000
mean,3.276592e+10,0.000200,253.575000,1.203666,1.045142,32765.624454,6.823635,0.133082,NaN,14.148148,...,5.700908,0.009909,1.767261e+05,66965.894623,2013.231393,46903.723724,0.778056,0.0,15.191243,15.245818
std,1.482121e+10,0.014147,451.870228,0.919067,1.978966,14821.193840,1.579030,0.339667,NaN,107.947444,...,2.470606,0.099051,3.454840e+05,60930.862599,4.638434,25508.057923,0.415558,0.0,21.639440,20.368479
min,1.001020e+09,0.000000,-1.000000,0.000000,1.000000,1001.000000,1.000000,0.000000,NaN,-22.000000,...,1.000000,0.000000,0.000000e+00,0.000000,2005.000000,1002.000000,0.000000,0.0,0.000414,0.000414
25%,1.901995e+10,0.000000,3.500000,1.000000,1.000000,19019.000000,6.000000,0.000000,NaN,-1.250000,...,7.000000,0.000000,1.000000e+05,30000.000000,2009.000000,27330.000000,1.000000,0.0,2.377668,2.598400
50%,3.702703e+10,0.000000,45.500000,1.000000,1.000000,37027.000000,7.000000,0.000000,NaN,0.000000,...,7.000000,0.000000,1.933000e+05,60000.000000,2013.000000,44558.000000,1.000000,0.0,6.267607,7.012343
75%,4.716502e+10,0.000000,218.000000,2.000000,1.000000,47165.000000,8.000000,0.000000,NaN,1.000000,...,7.000000,0.000000,2.500000e+05,100000.000000,2017.000000,75073.500000,1.000000,0.0,17.555267,18.874606
max,5.604198e+10,1.000000,2710.000000,4.000000,189.000000,56041.000000,10.000000,1.000000,NaN,796.000000,...,7.000000,1.000000,4.725000e+07,500000.000000,2021.000000,99224.000000,1.000000,0.0,100.000000,100.000000


In [8]:
df = pd.read_csv(output_path)
# Remove decimal and trailing zeros by converting to int then to string
df['censusTract'] = df['censusTract'].astype(float).astype('Int64').astype(str)

df.to_csv(output_path, index=False)

print("Cleaned censusTract column and replaced CSV.")


Cleaned censusTract column and replaced CSV.


In [9]:
#Get the counts of matching events
def getcountsofmatcheddata_dask(nfip_path, lsr_path):
    dtypes = {
        "QUALIFIER": "object",
        "FIPS": "object",               
        "VALID": "object",             
    }

    # Load with Dask using correct dtypes
    df_nfip = dd.read_csv(nfip_path, assume_missing=True)
    df_lsr = dd.read_csv(lsr_path, dtype=dtypes, assume_missing=True)

    # Preprocessing: clean and pad FIPS/censusTract
    df_nfip = df_nfip[df_nfip['censusTract'].notnull()]
    df_nfip['censusTract'] = df_nfip['censusTract'].astype(float).astype('Int64').astype(str).str.zfill(11)
    df_lsr['FIPS'] = df_lsr['FIPS'].astype(str).str.zfill(11)

    # Convert date columns
    df_nfip['dateOnly'] = dd.to_datetime(df_nfip['dateOfLoss'], errors='coerce').dt.normalize()
    df_lsr['LSR_dateOnly'] = dd.to_datetime(
        df_lsr['VALID'].astype(str).str[:8], format="%Y%m%d", errors='coerce'
    ).dt.normalize()

    # Filter valid NFIP rows
    df_nfip = df_nfip[(df_nfip['damagePercent'].notnull()) & (df_nfip['dateOnly'].notnull())]

    # Bring Dask to Pandas for optimized broadcasting
    df_nfip = df_nfip[['censusTract', 'dateOnly']].compute()
    df_lsr = df_lsr[['FIPS', 'LSR_dateOnly']].compute()

    print(f"NFIP valid rows: {len(df_nfip)}")
    print(f"LSR total rows: {len(df_lsr)}")

    results = []

    # Group LSR by FIPS for fast lookups
    lsr_grouped = df_lsr.groupby('FIPS')

    for delta in [0, 1, 2, 5, 10, 15, 20]:
        matched_count = 0

        # Iterate over chunks (or parallelize later)
        for _, row in df_nfip.iterrows():
            fips = row['censusTract']
            date = row['dateOnly']

            if fips not in lsr_grouped.groups:
                continue

            lsr_sub = lsr_grouped.get_group(fips)

            if delta == 0:
                matched = lsr_sub[lsr_sub['LSR_dateOnly'] == date]
            else:
                matched = lsr_sub[
                    (lsr_sub['LSR_dateOnly'] >= date - pd.Timedelta(days=delta)) &
                    (lsr_sub['LSR_dateOnly'] <= date + pd.Timedelta(days=delta))
                ]

            matched_count += len(matched)

        print(f"±{delta:>2} days → Matched LSR records: {matched_count}")
        results.append((delta, matched_count))

    return pd.DataFrame(results, columns=["±Days", "Matched_LSR_Records"])

In [10]:
getcountsofmatcheddata_dask(output_path, LSR)

NFIP valid rows: 36381
LSR total rows: 237063
± 0 days → Matched LSR records: 2370
± 1 days → Matched LSR records: 4144
± 2 days → Matched LSR records: 4446
± 5 days → Matched LSR records: 4858
±10 days → Matched LSR records: 5411
±15 days → Matched LSR records: 5834
±20 days → Matched LSR records: 6202


,±Days,Matched_LSR_Records
0,0,2370
1,1,4144
2,2,4446
3,5,4858
4,10,5411
5,15,5834
6,20,6202


In [11]:
def transfer_damagepercent(nfip_path, lsr_path, output_path, day_range=1):
    lsr_dtypes = {
        'FIPS': 'object',
        'VALID': 'object',
        'QUALIFIER': 'object' 
    }

    # Load datasets with Dask
    df_nfip = dd.read_csv(nfip_path, assume_missing=True)
    df_lsr = dd.read_csv(lsr_path, dtype=lsr_dtypes, assume_missing=True)

    # Filter and clean NFIP
    df_nfip = df_nfip[df_nfip['censusTract'].notnull()]
    df_nfip['censusTract'] = df_nfip['censusTract'].astype(float).astype('Int64').astype(str).str.zfill(11)
    df_nfip['dateOnly'] = dd.to_datetime(df_nfip['dateOfLoss'], errors='coerce').dt.normalize()
    df_nfip = df_nfip[df_nfip['damagePercent'].notnull() & df_nfip['dateOnly'].notnull()]
    df_nfip = df_nfip[['censusTract', 'dateOnly', 'damagePercent']].compute()

    # Clean LSR
    df_lsr['FIPS'] = df_lsr['FIPS'].astype(str).str.zfill(11)
    df_lsr['LSR_dateOnly'] = dd.to_datetime(
        df_lsr['VALID'].astype(str).str[:8], format="%Y%m%d", errors='coerce'
    ).dt.normalize()

    df_lsr = df_lsr.compute()
    df_lsr['damagePercent'] = pd.NA

    print(f"NFIP valid rows: {len(df_nfip)}")
    print(f"LSR total rows: {len(df_lsr)}")

    # Expand NFIP by ±day_range
    if day_range == 0:
        nfip_expanded = df_nfip.copy()
    else:
        offsets = range(-day_range, day_range + 1)
        nfip_expanded = pd.DataFrame([
            {'censusTract': row['censusTract'],
             'dateOnly': row['dateOnly'] + pd.Timedelta(days=offset),
             'damagePercent': row['damagePercent']}
            for _, row in df_nfip.iterrows()
            for offset in offsets
        ])

    # Merge LSR with expanded NFIP by FIPS and Date
    merged = pd.merge(
        df_lsr,
        nfip_expanded,
        left_on=['FIPS', 'LSR_dateOnly'],
        right_on=['censusTract', 'dateOnly'],
        how='left'
    )

    merged['damagePercent'] = merged['damagePercent_x'].combine_first(merged['damagePercent_y'])
    merged.drop(columns=['censusTract', 'dateOnly', 'damagePercent_x', 'damagePercent_y', 'LSR_dateOnly'], inplace=True)

    # Save output
    merged.to_csv(output_path, index=False)
    print(f"Exported LSR with damagePercent using ±{day_range} day range to:\n{output_path}")


In [12]:
#Get the geodataframe having the damage percentage
transfer_damagepercent(output_path, LSR, out_dir, day_range=1)

NFIP valid rows: 36381
LSR total rows: 237063


/tmp/ipykernel_1808/986435165.py:53: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  merged['damagePercent'] = merged['damagePercent_x'].combine_first(merged['damagePercent_y'])


Exported LSR with damagePercent using ±1 day range to:
../data/CONUS/LSR_pluvial_CONUS_X_damagepercent.csv


In [13]:
df_outdir = pd.read_csv(out_dir)
df_outdir.describe()

/tmp/ipykernel_1808/702973804.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_outdir = pd.read_csv(out_dir)


,VALID,LAT,LON,MAG,OBJECTID,Join_Count,TARGET_FID,STATE_FIPS,COUNTY_FIPS,STCOFIPS,TRACT_FIPS,POPULATION,POP_SQMI,SQMI,POPULATION_2020,POP20_SQMI,damagePercent
count,2.376010e+05,237601.000000,237601.000000,171017.000000,237601.000000,237601.000000,237601.000000,237597.000000,237597.000000,237597.000000,237597.000000,237597.000000,237597.000000,237597.000000,237597.000000,237597.000000,4144.000000
mean,2.016372e+11,39.766937,-91.661802,2.199409,118522.766996,0.999983,118522.766996,27.916379,88.451782,28004.831197,381795.103486,3949.952339,1105.976251,147.014353,3925.802569,1096.300645,20.780139
std,4.626019e+08,4.086102,11.801540,3.368137,68451.295580,0.004103,68451.295580,13.852570,84.117023,13859.650815,428509.126537,1663.496982,2818.022385,443.382590,1596.832195,2796.650212,25.921894
min,2.005010e+11,25.480000,-124.500000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1001.000000,100.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.058180
25%,2.013071e+11,37.600000,-96.210000,1.180000,59183.000000,1.000000,59183.000000,18.000000,31.000000,18049.000000,10300.000000,2790.000000,25.800000,3.590000,2798.000000,26.100000,3.805160
50%,2.017072e+11,40.280000,-89.640000,1.780000,118519.000000,1.000000,118519.000000,26.000000,73.000000,26103.000000,80100.000000,3743.000000,132.100000,30.140000,3743.000000,131.700000,9.213192
75%,2.020072e+11,42.550000,-84.630000,2.750000,177813.000000,1.000000,177813.000000,39.000000,123.000000,39017.000000,950400.000000,4902.000000,1190.700000,131.590000,4872.000000,1180.200000,28.353152
max,2.024123e+11,49.000000,-66.990000,1017.000000,237063.000000,1.000000,237063.000000,56.000000,840.000000,56043.000000,989300.000000,37577.000000,210233.300000,8989.530000,37892.000000,213333.300000,100.000000


**Get the selected LSR incidents only for 1 day window**

In [14]:
def transfer_damagepercent_matched_only(nfip_path, lsr_path, output_path, day_range=1):
    import dask.dataframe as dd
    import pandas as pd

    lsr_dtypes = {
        'FIPS': 'object',
        'VALID': 'object',
        'QUALIFIER': 'object' 
    }

    # Load datasets with Dask
    df_nfip = dd.read_csv(nfip_path, assume_missing=True)
    df_lsr = dd.read_csv(lsr_path, dtype=lsr_dtypes, assume_missing=True)

    # Clean and filter NFIP
    df_nfip = df_nfip[df_nfip['censusTract'].notnull()]
    df_nfip['censusTract'] = df_nfip['censusTract'].astype(float).astype('Int64').astype(str).str.zfill(11)
    df_nfip['dateOnly'] = dd.to_datetime(df_nfip['dateOfLoss'], errors='coerce').dt.normalize()
    df_nfip = df_nfip[df_nfip['damagePercent'].notnull() & df_nfip['dateOnly'].notnull()]
    df_nfip = df_nfip[['censusTract', 'dateOnly', 'damagePercent']].compute()

    # Clean LSR
    df_lsr['FIPS'] = df_lsr['FIPS'].astype(str).str.zfill(11)
    df_lsr['LSR_dateOnly'] = dd.to_datetime(
        df_lsr['VALID'].astype(str).str[:8], format="%Y%m%d", errors='coerce'
    ).dt.normalize()
    df_lsr = df_lsr.compute()
    df_lsr['damagePercent'] = pd.NA

    print(f"NFIP valid rows: {len(df_nfip)}")
    print(f"LSR total rows: {len(df_lsr)}")

    # Expand NFIP by ±day_range
    if day_range == 0:
        nfip_expanded = df_nfip.copy()
    else:
        offsets = range(-day_range, day_range + 1)
        nfip_expanded = pd.DataFrame([
            {'censusTract': row['censusTract'],
             'dateOnly': row['dateOnly'] + pd.Timedelta(days=offset),
             'damagePercent': row['damagePercent']}
            for _, row in df_nfip.iterrows()
            for offset in offsets
        ])

    # Merge LSR with expanded NFIP
    merged = pd.merge(
        df_lsr,
        nfip_expanded,
        left_on=['FIPS', 'LSR_dateOnly'],
        right_on=['censusTract', 'dateOnly'],
        how='left'
    )

    # Keep only matched rows
    matched = merged[merged['damagePercent_y'].notnull()].copy()

    # Use matched damage percent
    matched['damagePercent'] = matched['damagePercent_y']
    matched.drop(columns=['censusTract', 'dateOnly', 'damagePercent_x', 'damagePercent_y', 'LSR_dateOnly'], inplace=True)

    # Save only matched rows
    matched.to_csv(output_path, index=False)
    print(f"Exported ONLY MATCHED LSR rows (±{day_range} day window) to:\n{output_path}")


In [15]:
save_dir = Path("../data/CONUS/selected_LSR_conus_for1daywindow.csv")
transfer_damagepercent_matched_only(output_path, LSR, save_dir, day_range=1)

NFIP valid rows: 36381
LSR total rows: 237063
Exported ONLY MATCHED LSR rows (±1 day window) to:
../data/CONUS/selected_LSR_conus_for1daywindow.csv


**Assigning the NextGen catchment ID to all the LSR incidents**

In [16]:
#Nextgen directory
ngen_dir = Path("../data/conus_nextgenHF.gpkg")

In [17]:
#Assigning catchments for all LSR incidents
def ngen_cat(csv_path, geopkg_path, output_path, layer_name="divides", 
                               lat_col="LAT", lon_col="LON", catchment_col="divide_id"):
    """
    Assigns nextgen catchments to points in a CSV based on a polygon layer in a GeoPackage.

    Parameters:
        csv_path (str or Path): Path to the input CSV with latitude and longitude columns.
        geopkg_path (str or Path): Path to the GeoPackage file.
        layer_name (str): Name of the polygon layer (default: "divides").
        lat_col (str): Name of the latitude column in CSV (default: "latitude").
        lon_col (str): Name of the longitude column in CSV (default: "longitude").
        catchment_col (str): Name of the catchment ID or name column in the divides layer (default: "name").

    Returns:
        pd.DataFrame: Original CSV with an added 'nextgen_catchment' column.
    """
    df = pd.read_csv(csv_path)
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf_points = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

    # Load divides layer
    gdf_divides = gpd.read_file(geopkg_path, layer=layer_name)

    # Match CRS
    if gdf_points.crs != gdf_divides.crs:
        gdf_points = gdf_points.to_crs(gdf_divides.crs)
        
    joined = gpd.sjoin(gdf_points, gdf_divides[[catchment_col, "geometry"]], 
                       how="left", predicate="within")
    joined_clean = joined[~joined.index.duplicated(keep="first")]
    df['nextgen_catchment'] = joined_clean[catchment_col].reindex(df.index)

    # Save to file
    df.to_csv(output_path, index=False)
    print(f"Saved with nextgen_catchment to: {output_path}")

In [18]:
ngen_cat(csv_path=save_dir, geopkg_path = ngen_dir, output_path = "../data/CONUS/selected_LSR_conus_for1daywindow_NgenCat.csv")

Saved with nextgen_catchment to: ../data/CONUS/selected_LSR_conus_for1daywindow_NgenCat.csv


In [19]:
#Get rid with the same day multiple LSR information for a single NextGen Catchment
def filter_events(csv_path, output_path):
    df = pd.read_csv(csv_path)

    # Parse datetime column
    df['VALID2'] = pd.to_datetime(df['VALID2'], errors='coerce')

    # Create date-only column 
    df['event_date'] = df['VALID2'].dt.date

    def select_event(group):
        event_types = set(group['TYPETEXT'].str.upper())
        if 'FLASH FLOOD' in event_types:
            filtered = group[group['TYPETEXT'].str.upper() == 'FLASH FLOOD']
        elif 'HEAVY RAIN' in event_types:
            filtered = group[group['TYPETEXT'].str.upper() == 'HEAVY RAIN']
        else:
            return None  # if neither event type is present
        return filtered.sort_values('VALID2').head(1)

    filtered_df = (
        df.groupby(['nextgen_catchment', 'event_date'], group_keys=False)
        .apply(select_event)
        .reset_index(drop=True)
    )

    # Save to CSV
    filtered_df.to_csv(output_path, index=False)
    print(f"Saved filtered results to {output_path}")

In [20]:
filter_events("../data/CONUS/selected_LSR_conus_for1daywindow_NgenCat.csv", "../data/CONUS/selected_LSR_conus_for1daywindow_NgenCat_filtered.csv")

Saved filtered results to ../data/CONUS/selected_LSR_conus_for1daywindow_NgenCat_filtered.csv


/tmp/ipykernel_1808/1509929466.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(select_event)


**Exploring the dataset about having multiple incidents in the same catchment**

In [14]:
df = pd.read_csv("../data/CONUS/selected_LSR_conus_for1daywindow_NgenCat_filtered.csv")

# Counting unique damagePercent per catchment
damage_counts = (
    df.groupby("nextgen_catchment")["damagePercent"]
    .nunique()
    .reset_index(name="unique_damage_incidents")
)

# Filter to catchments with more than 5 unique damage records
catchments_gt5 = damage_counts[damage_counts["unique_damage_incidents"] > 1]

# Number of such catchments
num_catchments_gt5 = catchments_gt5.shape[0]

num_unique_catchments = df["nextgen_catchment"].nunique()
print(f"Total unique catchments with at least one damage record: {num_unique_catchments}")


print(f"Number of catchments with more than 2 distinct damage records: {num_catchments_gt5}")
matching_events_df = df[df["nextgen_catchment"].isin(catchments_gt5["nextgen_catchment"])]
print(f"Total number of events from catchments with >1 unique damagePercent values: {matching_events_df.shape[0]}")

print("Catchment IDs:")
print(catchments_gt5["nextgen_catchment"].tolist())

Total unique catchments with at least one damage record: 2321
Number of catchments with more than 2 distinct damage records: 255
Total number of events from catchments with >1 unique damagePercent values: 595
Catchment IDs:
['cat-100224', 'cat-1010263', 'cat-1021677', 'cat-1023764', 'cat-102612', 'cat-103066', 'cat-105032', 'cat-105049', 'cat-105101', 'cat-105663', 'cat-105836', 'cat-1077437', 'cat-1077455', 'cat-1077921', 'cat-1078797', 'cat-1078883', 'cat-1078884', 'cat-1078889', 'cat-1078901', 'cat-1078916', 'cat-1079044', 'cat-1079156', 'cat-1079314', 'cat-1079621', 'cat-1080803', 'cat-1081184', 'cat-1081895', 'cat-1088778', 'cat-1090024', 'cat-1091210', 'cat-1092307', 'cat-1092590', 'cat-1093260', 'cat-1093967', 'cat-1094076', 'cat-1094439', 'cat-1097193', 'cat-109748', 'cat-1098522', 'cat-1098532', 'cat-1100007', 'cat-1100010', 'cat-1100111', 'cat-1100508', 'cat-1100659', 'cat-1100668', 'cat-1100834', 'cat-1103277', 'cat-1106175', 'cat-1106551', 'cat-1111774', 'cat-1111984', 'cat

In [17]:
df = pd.read_csv("../data/CONUS/selected_LSR_conus_for1daywindow_NgenCat_filtered.csv")

# Count unique damagePercent values per catchment
damage_counts = (
    df.groupby("nextgen_catchment")["damagePercent"]
    .nunique()
    .reset_index(name="unique_damage_incidents")
)

# Keep only catchments with more than 2 unique damage records
catchments_gt2 = damage_counts[damage_counts["unique_damage_incidents"] > 1]

# Filter original dataframe to include only those catchments
filtered_df = df[df["nextgen_catchment"].isin(catchments_gt2["nextgen_catchment"])]

# Save the filtered data to CSV
filtered_df.to_csv("../data/CONUS/Filtered_LSR_withNexgenCatchmentsgt1.csv", index=False)


In [18]:
#get the unique catchments
def get_unique_catchments(csv_path):
    df = pd.read_csv(csv_path)
    unique_catchments = df['nextgen_catchment'].dropna().unique()
    unique_catchments = sorted(unique_catchments)  # optional sorting
    count = len(unique_catchments)
    
    print(f"Total unique catchments: {count}")
    return unique_catchments

In [19]:
get_unique_catchments("../data/CONUS/Filtered_LSR_withNexgenCatchmentsgt1.csv")

Total unique catchments: 255


['cat-100224',
 'cat-1010263',
 'cat-1021677',
 'cat-1023764',
 'cat-102612',
 'cat-103066',
 'cat-105032',
 'cat-105049',
 'cat-105101',
 'cat-105663',
 'cat-105836',
 'cat-1077437',
 'cat-1077455',
 'cat-1077921',
 'cat-1078797',
 'cat-1078883',
 'cat-1078884',
 'cat-1078889',
 'cat-1078901',
 'cat-1078916',
 'cat-1079044',
 'cat-1079156',
 'cat-1079314',
 'cat-1079621',
 'cat-1080803',
 'cat-1081184',
 'cat-1081895',
 'cat-1088778',
 'cat-1090024',
 'cat-1091210',
 'cat-1092307',
 'cat-1092590',
 'cat-1093260',
 'cat-1093967',
 'cat-1094076',
 'cat-1094439',
 'cat-1097193',
 'cat-109748',
 'cat-1098522',
 'cat-1098532',
 'cat-1100007',
 'cat-1100010',
 'cat-1100111',
 'cat-1100508',
 'cat-1100659',
 'cat-1100668',
 'cat-1100834',
 'cat-1103277',
 'cat-1106175',
 'cat-1106551',
 'cat-1111774',
 'cat-1111984',
 'cat-1121208',
 'cat-1121664',
 'cat-1122758',
 'cat-1123000',
 'cat-1123314',
 'cat-1123351',
 'cat-1123606',
 'cat-1123628',
 'cat-1125753',
 'cat-114960',
 'cat-115994',
 'c